In [1]:
import yaml 
with open('/root/graduation_thetis/causal-bert-pytorch/run/result/seed_everything_appliance_contain_text_with_aug.yaml','r') as yml:
    yaml_data = yaml.safe_load(yml)
print(yaml_data)

{'exp1': {'ATE': -0.12891103841863105, 'batch_size': 32, 'epochs': 3, 'model_name': 'timm/eva02_tiny_patch14_224.mim_in22k', 'seed': 42}, 'exp10': {'ATE': -0.0615148858908385, 'batch_size': 32, 'epochs': 3, 'model_name': 'timm/eva02_tiny_patch14_224.mim_in22k', 'seed': 2021}, 'exp11': {'ATE': -0.08696908395462861, 'batch_size': 32, 'epochs': 3, 'model_name': 'timm/eva02_tiny_patch14_224.mim_in22k', 'seed': 2223}, 'exp12': {'ATE': -0.0500756520463092, 'batch_size': 32, 'epochs': 3, 'model_name': 'timm/eva02_tiny_patch14_224.mim_in22k', 'seed': 2425}, 'exp13': {'ATE': -0.08549198056391721, 'batch_size': 32, 'epochs': 3, 'model_name': 'timm/eva02_tiny_patch14_224.mim_in22k', 'seed': 2627}, 'exp14': {'ATE': -0.11727697358915659, 'batch_size': 32, 'epochs': 3, 'model_name': 'timm/eva02_tiny_patch14_224.mim_in22k', 'seed': 2829}, 'exp15': {'ATE': -0.07273807236830312, 'batch_size': 32, 'epochs': 3, 'model_name': 'timm/eva02_tiny_patch14_224.mim_in22k', 'seed': 3031}, 'exp16': {'ATE': -0.0479

In [2]:
import numpy as np
eva_ate = []
vit_ate = []
eff_ate = []
for k,v in yaml_data.items():
    if v['model_name'] == 'timm/eva02_tiny_patch14_224.mim_in22k':
        eva_ate.append(v['ATE'])
    if v['model_name'] == 'timm/efficientvit_m2.r224_in1k':
        eff_ate.append(v['ATE'])
    if v['model_name'] == 'timm/vit_base_patch32_clip_224.laion2b_ft_in12k_in1k':
        vit_ate.append(v['ATE'])

print("eva_ate", np.mean(eva_ate),np.std(eva_ate))
print("vit_ate", np.mean(vit_ate),np.std(vit_ate))
print("eff_ate", np.mean(eff_ate),np.std(eff_ate))

eva_ate -0.07374221698171317 0.02975074780237661
vit_ate -0.03828088064867551 0.054364578057533856
eff_ate -0.025481311331780408 0.0869782046963629


In [3]:
from collections import defaultdict
import numpy as np
import pandas as pd
def ATE_unadjusted(T, Y):
    x = defaultdict(list)
    for t, y in zip(T, Y):
        x[t].append(y)
    T0 = np.mean(x[0])
    T1 = np.mean(x[1])
    return T0 - T1

def ATE_adjusted(C, T, Y):
    x = defaultdict(list)
    for c, t, y in zip(C, T, Y):
        x[c, t].append(y)

    C0_ATE = np.mean(x[0,0]) - np.mean(x[0,1])
    C1_ATE = np.mean(x[1,0]) - np.mean(x[1,1])
    return np.mean([C0_ATE, C1_ATE])


if __name__ == "__main__":
    df = pd.read_csv("/root/graduation_thetis/causal-bert-pytorch/input/Appliances_preprocess_contains_text_1122.csv")
    treatment = "light_or_dark"
    confounder = "contains_text"
    outcome = "outcome"
    print("ATE_unadjusted: ", ATE_unadjusted(df[treatment], df[outcome]))
    print("ATE_adjusted: ", ATE_adjusted(df[confounder], df[treatment],df[outcome]))

ATE_unadjusted:  -0.06485416818955525
ATE_adjusted:  0.03791337506722073


In [5]:
df["light_or_dark"].value_counts()

light_or_dark
1    4454
0    1462
Name: count, dtype: int64